This notebook contains the code that I used to scrape the petfinders.my website for pets that were put for adoption

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from fake_useragent import UserAgent
import random
import time


In [2]:
def scrape_adopted_urls(url,runtime_regex):
    '''(url string ---> list of adopted pets urls)
    
    This function takes a url of a page and returns the links to
    the adopted peets on that page
    
    '''
    adopted_urls_per_page = []
    
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}

    response = requests.get(url, headers = user_agent)
    print(response.status_code)

    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    
    maincontent = soup.find('div', id = 'maincontent')
    list_adopted = maincontent.find_all('font', string = runtime_regex)
    list_adopted.pop(0) # the first row contains the 'Adopted' but not a pet
    for ix, entry in enumerate(list_adopted):
        url = list_adopted[ix].parent.parent.parent.find('a').get("href")
        adopted_urls_per_page.append(url)
    return adopted_urls_per_page

In [9]:
def append_scraped_urls(num_pages):
    ''' (integer ---> list) 
    
    This function takes in the number of pages to be scraped
    and returns the list of the urls
    '''
    url_start = 'https://www.petfinder.my/listings.php?page='
    url_end = '&sorttype=&sortage=&sortadopt=1&sortfield=&sortbreed1=&sortbreed2=&sortstate=&sortquery='
    
    runtime_regex = re.compile('Adopted')
    urls_adopted_list = []
    for i in range(1,num_pages+1): 
        url = url_start+str(i)+url_end
        adopted_urls_per_page = scrape_adopted_urls(url,runtime_regex)
        #print(adopted_urls_per_page)
        #urls_adopted_list.append(adopted_urls_per_page)
        urls_adopted_list = urls_adopted_list + adopted_urls_per_page
        time.sleep(30+1.5*random.random())

    return urls_adopted_list


    

In [10]:
def scrape_pet_page(pet_url):
    '''(string ---> dictionary)
    
    This function takes the url to a pet information page
    and scrapes all the information into a dictionary
    
    '''
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}
    
    response = requests.get(pet_url, headers = user_agent)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')

    maincontent = soup.find('div', id = 'maincontent')

    pet = maincontent.find('td', string = re.compile('Dog|Cat'))
    
    if pet != None:
        pet_profile = pet.parent.parent

        pet_profile_dictionary = {}
        for i in pet_profile.find_all('tr'):
            line = i.get_text(separator='|', strip=True).split('|')
            if len(line)>1:
                pet_profile_dictionary[line[0]] = line[1:]
        pet_discription = maincontent.find('div', id = 'pet_desc').text
        pet_profile_dictionary['Pet Discription'] = pet_discription    
        return pet_profile_dictionary
    else:
        return {}



In [11]:
def accumulate_pets(pet_urls):
    '''This function takes a list of urls that contain pet profiles 
    and returns a list of dictionaries where each dictionary
    is a pet profile
    '''
    pet_profiles = []
    for url in pet_urls:
        pet_profile_dict = scrape_pet_page(url)
        if pet_profile_dict != {}:
            pet_profiles.append(pet_profile_dict)
            time.sleep(30+1.5*random.random())

    return pet_profiles

In [12]:
# getting urls to pet profiles and saving them to a file
pet_urls = append_scraped_urls(2)

with open('pet_urls.txt', 'w') as f:
    for item in pet_urls:
        f.write("%s\n" % item)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/opt/anaconda3/envs/metis/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/opt/anaconda3/envs/metis/lib/python3.7/http/client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/anaconda3/envs/metis/lib/python3.7/http/client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/anaconda3/envs/metis/lib/python3.7/http/client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/anaconda3/envs/metis/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/opt/anaconda3/envs/metis/lib/python3.7/http/client.py", line 966, in send
    self.connect()
  File "/opt/anaconda3/envs/m

200
200


In [13]:
# getting the pet profiles and saving them into a file
All_pets = accumulate_pets(pet_urls)
with open('data.txt', 'w') as f:
    for item in All_pets:
        f.write("%s\n" % item)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
